# Creating Response Dimenion Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.gold.dim_response
(
  response_id STRING NOT NULL PRIMARY KEY,
  complaint_number INT,
  company_response_to_consumer STRING,
  company_public_response STRING,
  start_date DATE,
  end_date DATE
)
USING DELTA
LOCATION 'abfss://gold@rcmadls10dev.dfs.core.windows.net/dim_response';

In [0]:
%sql
-- select category_id, count(category_id) from databricks_catalog.gold.dim_category 
-- group by category_id
-- having count(category_id) > 1
-- ;

-- drop table databricks_catalog.gold.dim_category;

-- truncate table databricks_catalog.gold.dim_category;

In [0]:
%sql
WITH deduped_source AS (
  SELECT
    response_id,
    FIRST(complaint_id) as complaint_id,
    FIRST(company_response_to_consumer) as company_response_to_consumer,
    FIRST(company_public_response) as company_public_response
  FROM databricks_catalog.silver.complaints
  WHERE response_id IS NOT NULL
  GROUP BY response_id
),

records_to_update AS (
  SELECT
    tgt.response_id,
    src.complaint_id,
    src.company_response_to_consumer,
    src.company_public_response
  FROM databricks_catalog.gold.dim_response tgt
  INNER JOIN deduped_source src
    ON tgt.response_id = src.response_id
  WHERE tgt.end_date = to_date('2099-12-31') AND (
    tgt.company_response_to_consumer <> src.company_response_to_consumer OR
    tgt.company_public_response <> src.company_public_response OR
    tgt.complaint_number <> src.complaint_id
  )
)

UPDATE
 databricks_catalog.gold.dim_response
SET
  end_date = current_date()
WHERE response_id IN (SELECT response_id FROM records_to_update)
AND end_date = to_date('2099-12-31');

In [0]:
%sql
WITH deduped_source AS (
  SELECT
    response_id,
    FIRST(complaint_id) as complaint_id,
    FIRST(company_response_to_consumer) as company_response_to_consumer,
    FIRST(company_public_response) as company_public_response
  FROM databricks_catalog.silver.complaints
  WHERE response_id IS NOT NULL
  GROUP BY response_id
)

INSERT INTO databricks_catalog.gold.dim_response (response_id, complaint_number, company_response_to_consumer, company_public_response, start_date, end_date)
SELECT
  src.response_id AS response_id,
  src.complaint_id AS complaint_number,
  src.company_response_to_consumer AS company_response_to_consumer,
  src.company_public_response AS company_public_response,
  current_date() AS start_date,
  to_date('2099-12-31') AS end_date
FROM
  deduped_source src
LEFT JOIN
  databricks_catalog.gold.dim_response tgt
ON
  src.response_id = tgt.response_id AND tgt.end_date = to_date('2099-12-31')
WHERE
  tgt.response_id IS NULL -- new response_id
  OR (
    tgt.complaint_number <> src.complaint_id OR
    tgt.company_response_to_consumer <> src.company_response_to_consumer OR
    tgt.company_public_response <> src.company_public_response
  );


In [0]:
%sql
select * from databricks_catalog.gold.dim_category;